In [ ]:
from scippneutron.tof import unwrap
from scippneutron.tof.unwrap import (
    RawData,
    PulsePeriod,
    SourceTimeRange,
    SourceWavelengthRange,
    Choppers,
    SourceChopperName,
    L1,
    L2,
    TofData,
    PulseStride,
    FirstPulseTime,
)

## Normal

In [ ]:
import scipp as sc
import sciline as sl

pl = sl.Pipeline(unwrap.providers_normal_mode)
pl[RawData] = sc.DataArray(data=sc.ones(dims=['event_time_offset'], shape=[1000]))
pl[PulsePeriod] = sc.scalar(71.0, unit='ms')
pl[SourceTimeRange] = sc.scalar(0.0, unit='ms'), sc.scalar(4.0, unit='ms')
pl[SourceWavelengthRange] = sc.scalar(0.0, unit='angstrom'), sc.scalar(
    10.0, unit='angstrom'
)
pl[Choppers] = {}
pl[SourceChopperName] = 'PSC1'
pl[L1] = sc.scalar(20.0, unit='m')
pl[L2] = sc.scalar(1.0, unit='m')
pl.get(TofData).visualize()

## Pulse-skipping mode

Note that chopper open- and close- times must be relative to the frame start, so in practice creating the choppers will also require info such as `FirstPulseTime` or something related.

In [ ]:
import sciline as sl

pl = sl.Pipeline(unwrap.providers_pulse_skipping_mode)
pl[RawData] = sc.DataArray(data=sc.ones(dims=['event_time_offset'], shape=[1000]))
pl[PulsePeriod] = sc.scalar(71.0, unit='ms')
pl[PulseStride] = 2
pl[SourceTimeRange] = sc.scalar(0.0, unit='ms'), sc.scalar(4.0, unit='ms')
pl[SourceWavelengthRange] = sc.scalar(0.0, unit='angstrom'), sc.scalar(
    10.0, unit='angstrom'
)
pl[Choppers] = {}
pl[SourceChopperName] = 'PSC1'
pl[L1] = sc.scalar(20.0, unit='m')
pl[L2] = sc.scalar(1.0, unit='m')
pl[FirstPulseTime] = sc.scalar(0.0, unit='ms')
# pl.get(unwrap.WrappedTimeOffset).visualize()
# pl.get(unwrap.TimeOffset).visualize()
pl.get(TofData).visualize()

## WFM

In [ ]:
import sciline as sl

pl = sl.Pipeline(unwrap.providers_wfm)
pl[RawData] = sc.DataArray(data=sc.ones(dims=['event_time_offset'], shape=[1000]))
pl[PulsePeriod] = sc.scalar(71.0, unit='ms')
pl[SourceTimeRange] = sc.scalar(0.0, unit='ms'), sc.scalar(4.0, unit='ms')
pl[SourceWavelengthRange] = sc.scalar(0.0, unit='angstrom'), sc.scalar(
    10.0, unit='angstrom'
)
pl[Choppers] = {}
pl[SourceChopperName] = 'PSC1'
pl[L1] = sc.scalar(20.0, unit='m')
pl[L2] = sc.scalar(1.0, unit='m')
# pl.get(unwrap.WrappedTimeOffset).visualize()
# pl.get(unwrap.TimeOffset).visualize()
pl.get(TofData).visualize()

In [ ]:
import scipp as sc

nevent = int(1e7)
npix = int(1e6)
table = sc.data.table_xyz(nevent)
da = table.bin(x=npix)
y_bins = sc.array(dims=['y'], values=[0.0, 0.1, 0.2, 0.4, 0.8, 1], unit='m')
hist = sc.DataArray(sc.ones(dims=['y'], shape=[5], unit='m'), coords={'y': y_bins})
# hist = sc.concat([hist]*npix, 'x')
lut = sc.lookup(hist, dim='y')

In [ ]:
%%time
y = da.bins.coords['y']
y -= lut[y]

In [ ]:
%%time
comps = da.bins.constituents
#comps['data'] = comps['data'].copy(deep=False)
comps['data'] = sc.DataArray(comps['data'].data, coords={'y':comps['data'].coords['y']})
tmp = sc.DataArray(sc.bins(**comps))
tmp.bins *= lut
tmp

In [ ]:
%%time
tmp = da.bin(y=y_bins)
tmp.bins.coords['y'] *= hist.data
tmp.bins.concat('y')